In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib
import seaborn as sns
import cartopy.crs as ccrs
import glob
from xskillscore import crps_gaussian
from scipy.stats import ttest_ind_from_stats as ttest
import pandas as pd
import os
import dataframe_image as dfi

In [2]:
#some utils:

def area_grid(lat, lon):
    """
    Calculate the area of each grid cell
    Area is in square meters
    -----------
    modified from
    https://towardsdatascience.com/the-correct-way-to-average-the-globe-92ceecd172b7
    Based on the function in
    https://github.com/chadagreene/CDT/blob/master/cdt/cdtarea.m
    """
    from numpy import meshgrid, deg2rad, gradient, cos
    from xarray import DataArray

    xlon, ylat = meshgrid(lon, lat)
    R = earth_radius(ylat)

    dlat = deg2rad(gradient(ylat, axis=0))
    dlon = deg2rad(gradient(xlon, axis=1))

    dy = dlat * R
    dx = dlon * R * cos(deg2rad(ylat))

    area = dy * dx

    xda = DataArray(
        area,
        dims=["lat", "lon"],
        coords={"lat": lat, "lon": lon},
        attrs={
            "long_name": "area_per_pixel",
            "description": "area per pixel",
            "units": "m^2",
        },
    )
    return xda

def earth_radius(lat):
    '''
    -----------
    Copied from
    https://towardsdatascience.com/the-correct-way-to-average-the-globe-92ceecd172b7
    WGS84: https://earth-info.nga.mil/GandG/publications/tr8350.2/tr8350.2-a/Chapter%203.pdf
    '''
    from numpy import deg2rad, sin, cos

    # define oblate spheroid from WGS84
    a = 6378137
    b = 6356752.3142
    e2 = 1 - (b**2/a**2)
    
    # convert from geodecic to geocentric
    # see equation 3-110 in WGS84
    lat = deg2rad(lat)
    lat_gc = np.arctan( (1-e2)*np.tan(lat) )

    # radius equation
    # see equation 3-107 in WGS84
    r = (
        (a * (1 - e2)**0.5) 
         / (1 - (e2 * np.cos(lat_gc)**2))**0.5 
        )

    return r


def lat_weighted_mean(x):
    weights = np.cos(np.deg2rad(x.lat))
    return x.weighted(weights).mean(['lat', 'lon'])

def lat_weighted_sum(x):
    weights = np.cos(np.deg2rad(x.lat))
    return x.weighted(weights).sum(['lat', 'lon'])

def normalize_data(data):
    #normalize data by mean and std
    mean = data.mean()
    std = data.std()
    data = (data - mean) / std
    return data, mean, std


#NRMSE:
def get_nrmse(truth, pred):
    weights = np.cos(np.deg2rad(truth.lat))
    return np.sqrt(((truth - pred)**2).weighted(weights).mean(['lat', 'lon'])).data / np.abs(truth.weighted(weights).mean(['lat','lon']).data)

#mean bias:
def get_bias(truth, pred):
    weights = np.cos(np.deg2rad(truth.lat))
    return (pred - truth).weighted(weights).mean(['lat', 'lon']).data

#crps:
def get_crps(truth, pred, std):
    weights = np.cos(np.deg2rad(truth.lat))
    return crps_gaussian(truth, pred, std, weights=weights).data

In [9]:
# create pandas dataframe of stats for Figure 2.
# Columns are:
# 0. experiment name
# 1. % of area of siginificant response in NorESM experiment (p_val<0.05 for last 10 years of transient experiments, p_val<0.05 for equilibrium experiments)
# 2. NRMSE for NorESM and AeroGP
# 3. CRPS for NorESM and AeroGP
# 4. Mean Bias

s =[]

for ii in glob.glob('../NorESM_stat_data/stats_fdc*.nc'):
    stats = xr.open_dataset(ii)

    pval = stats.fdr_adjusted_p_values

    #experiment name:
    exp = stats.attrs['experiment_id']

    #for equilibrium experiments p-values are 2D:
    if pval.ndim == 2:
        pval_area = (area_grid(stats.lat, stats.lon).where(pval<0.05)).sum()

    #for transient experiments p-values are 3D, we use last 20 years (first window):
    if pval.ndim == 3:
        pval_area = (area_grid(stats.lat, stats.lon).where(pval[0]<0.05)).sum()
    
    #percentage of significant area (where p<0.05):
    total_area = area_grid(stats.lat, stats.lon).sum()
    significant_area = (pval_area.values/total_area.values)*100

    #append to list:
    s.append(
        {
            'Experiment': exp,
            'NorESM mean sig. area (%)': significant_area,
    
        }
    )



#calculate NRMSE, CRPS and Bias for each experiment:
s2 = []

for ii in glob.glob('../training_data_v2/outputs*.nc'):
    
    #get NorESM target data:
    NorESM_out = xr.open_dataset(ii)
    exp_id = NorESM_out.attrs['experiment_id']
    print(ii)

    if exp_id == 'RAMIP T1: ssp126-ssp370':
        exp_id = 'RAMIP T1'

    #get corresponding AeroGP posterior:
    list_of_files = glob.glob('../logs/lr_1lin/' + exp_id + '/posterior_predictY_*.nc') # * means all if need specific format then *.csv
    #list_of_files = glob.glob('../logs/logs_' + exp_id + '/posterior_predictY_*.nc')
    latest_file = max(list_of_files, key=os.path.getctime)
    AeroGP_out = xr.open_dataset(latest_file)
    print(latest_file)
    
    #for transient experiments use mean of last 20 years:
    if exp_id == 'ssp245-aer': # use first 20 years:
        NorESM_out_mean = NorESM_out['tas_diff'].isel(year=slice(0,20)).mean('year')
        AeroGP_out_mean = AeroGP_out['posterior'].isel(year=slice(0,20)).mean('year')
        AeroGP_out_std = np.sqrt(np.power(AeroGP_out['posterior std'].isel(year=slice(0,20)),2).mean('year'))
        NorESM_glb_mean = lat_weighted_mean(NorESM_out_mean).values
        AeroGP_glb_mean = lat_weighted_mean(AeroGP_out_mean).values

    elif NorESM_out.tas_diff.ndim == 3:
        NorESM_out_mean = NorESM_out['tas_diff'].isel(year=slice(-20,None)).mean('year')
        AeroGP_out_mean = AeroGP_out['posterior'].isel(year=slice(-20,None)).mean('year')
        AeroGP_out_std = np.sqrt(np.power(AeroGP_out['posterior std'].isel(year=slice(-20,None)),2).mean('year'))
        NorESM_glb_mean = lat_weighted_mean(NorESM_out_mean).values
        AeroGP_glb_mean = lat_weighted_mean(AeroGP_out_mean).values

    elif NorESM_out.tas_diff.ndim == 2:
        NorESM_out_mean = NorESM_out['tas_diff']
        AeroGP_out_mean = AeroGP_out['posterior']
        AeroGP_out_std = AeroGP_out['posterior std']
        NorESM_glb_mean = lat_weighted_mean(NorESM_out_mean).values
        AeroGP_glb_mean = lat_weighted_mean(AeroGP_out_mean).values

    #calculate NRMSE, CRPS and Bias:
    nrmse = get_nrmse(NorESM_out_mean, AeroGP_out_mean)
    crps = get_crps(NorESM_out_mean, AeroGP_out_mean, AeroGP_out_std)
    bias = get_bias(NorESM_out_mean, AeroGP_out_mean)


    #append to list:
    s2.append(
        {
            'Experiment': exp_id,
            'NRMSE': nrmse,
            'CRPS': crps,
            'Mean Bias': bias,
            r'ΔT' + ' NorESM (K)': NorESM_glb_mean,
            r'ΔT' + ' AeroGP (K)': AeroGP_glb_mean,
        }
    )

#merge the two lists:
df1 = pd.DataFrame(s)
df2 = pd.DataFrame(s2)
df = pd.merge(df1, df2, on='Experiment')


../training_data_v2\outputs_0xEA_SO2.nc
../logs/lr_1lin/0xEA SO2 only\posterior_predictY_0xEA_SO2_20250325-002212.nc
../training_data_v2\outputs_0xEU_SO2.nc
../logs/lr_1lin/0xEU SO2 only\posterior_predictY_0xEU_SO2_20250325-013028.nc
../training_data_v2\outputs_0xNA_SO2.nc
../logs/lr_1lin/0xNA SO2 only\posterior_predictY_0xNA_SO2_20250325-043248.nc
../training_data_v2\outputs_0xSA_SO2.nc
../logs/lr_1lin/0xSA SO2 only\posterior_predictY_0xSA_SO2_20250325-071124.nc
../training_data_v2\outputs_10xEU_BC.nc
../logs/lr_1lin/10xEU BC only\posterior_predictY_10xEU_BC_20250325-015301.nc
../training_data_v2\outputs_10xNA_BC.nc
../logs/lr_1lin/10xNA BC only\posterior_predictY_10xNA_BC_20250325-045537.nc
../training_data_v2\outputs_10xSA_BC.nc
../logs/lr_1lin/10xSA BC only\posterior_predictY_10xSA_BC_20250325-073416.nc
../training_data_v2\outputs_10xSA_SO2.nc
../logs/lr_1lin/10xSA SO2 only\posterior_predictY_10xSA_SO2_20250325-075655.nc
../training_data_v2\outputs_5xEA_BC.nc
../logs/lr_1lin/5xEA B

In [10]:
exp_id

'ssp370-lowNTCF'

In [11]:
# some edits for display

#fix some experiment names for plotting table:
df.replace('ssp245-aer', 'DAMIP ssp245-aer', inplace=True)
df.replace('hist-aer', 'DAMIP hist-aer', inplace=True)

df = df.infer_objects().astype(np.float64, errors='ignore')

#add column for NorESM version:
df['NorESM version'] = np.where(df['Experiment'].str.contains('RAMIP') | df['Experiment'].str.contains('DAMIP') | df['Experiment'].str.contains('AeroGP') | df['Experiment'].str.contains('ssp370'), 'NorESM2', 'NorESM1')


In [12]:
df.sort_values(by='NRMSE').style\
    .background_gradient()\
    .format(precision=2)\
    .hide()

Experiment,NorESM mean sig. area (%),NRMSE,CRPS,Mean Bias,ΔT NorESM (K),ΔT AeroGP (K),NorESM version
RAMIP T1,68.08,0.37,0.14,-0.00,0.47,0.46,NorESM2
DAMIP hist-aer,77.41,0.43,0.20,0.18,-0.84,-0.66,NorESM2
DAMIP ssp245-aer,47.49,0.56,0.18,0.15,-0.58,-0.44,NorESM2
Global Anthro Removal,80.45,0.63,0.20,0.02,0.55,0.57,NorESM1
Global SO2 removal only,69.76,0.64,0.15,0.05,0.36,0.42,NorESM1
AeroGP2,52.24,0.66,0.14,-0.08,0.29,0.22,NorESM2
10xSA SO2 only,62.77,0.86,0.14,0.04,-0.17,-0.14,NorESM1
7xEU SO2 only,66.84,0.87,0.15,0.03,-0.25,-0.22,NorESM1
10xEU BC only,56.34,0.88,0.14,-0.07,0.16,0.09,NorESM1
ssp370-lowNTCF,30.32,0.93,0.14,0.05,0.19,0.24,NorESM2


In [ ]:
#save to csv:
df.to_csv('figure2_stats_lr.csv', index=False)

In [13]:
#save to png:
#dfi.export(df_styled,'Evaluation_stats_table.png')